In [1]:
import torch
import time
from snn_delays.snn import SNN
from snn_delays.utils.dataset_loader import DatasetLoader
from snn_delays.utils.train_utils import get_device, print_spike_info, propagate_batch, set_seed
from snn_delays.utils.visualization_utils import plot_raster
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Subset, DataLoader
from collections import OrderedDict
device = get_device()
from torch.utils.data import Dataset, DataLoader
from tonic import MemoryCachedDataset
# for reproducibility
set_seed(42)

# Custom dataset for digit concatenation from a filtered dataset
class SequentialMemoryRetrievalDataset(Dataset):
    def __init__(self, base_dataset, sequence_length, target_classes):
        self.base_dataset = base_dataset
        indices = [i for i, (img, label) in enumerate(base_dataset) if np.argmax(label) in target_classes]
        zero_indices = [i for i, (img, label) in enumerate(base_dataset) if np.argmax(label) == 0]
        
        self.filtered_dataset = Subset(base_dataset, indices)
        self.zeros_dataset = Subset(base_dataset, zero_indices)

        self.indices = list(range(len(self.filtered_dataset)))  # Indices of the base dataset
        self.zeros_indices = list(range(len(self.zeros_dataset)))

        self.target_classes = target_classes
        #self.pairs = [(i, j) for i in self.indices for j in self.indices]  # All possible pairs of indices
        self.sequence_length = sequence_length
        #self.pairs = list(product(self.indices, repeat=sequence_length))
        self.num_classes = len(target_classes) # 0 is not a class 
        self.total_combinations = self.num_classes ** 2

    def __len__(self):
        # Number of pairs
        #return len(self.pairs)
        return len(self.base_dataset)

    def __getitem__(self, idx):
        # Get the indices for the current pair
        #indices = self.pairs[idx]
        # Retrieve the images and labels from the base dataset

        images = []
        all_labels = []
        retrieval_labels = []

        for i in range(self.sequence_length-2):
            img, label = self.filtered_dataset[np.random.choice(self.indices)]
            images.append(img)
            all_labels.append(self.target_classes.index(np.argmax(label)))

        # inserts zeroes randomly ()
        idx_frst_zero = np.random.randint(self.sequence_length//2)
        img, _ = self.zeros_dataset[np.random.choice(self.zeros_indices)]
        images.insert(idx_frst_zero, img)
        retrieval_labels.append(all_labels[idx_frst_zero])

        idx_scnd_zero = np.random.randint(idx_frst_zero+2, self.sequence_length-1)
        img, _ = self.zeros_dataset[np.random.choice(self.zeros_indices)]
        images.insert(idx_scnd_zero, img)
        retrieval_labels.append(all_labels[idx_scnd_zero-1])

        # Concatenate the images along the width (you can adjust as needed)
        concatenated_img = np.concatenate(images, axis=0)
        
        # Concatenate the labels one-hot
        encoded_label = sum(l * (self.num_classes ** i) for i, l in enumerate(reversed(retrieval_labels)))
        concatenated_label = np.zeros(self.total_combinations)
        concatenated_label[encoded_label] = 1.0

        return concatenated_img, concatenated_label


c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on: cuda:0


In [6]:
from snn_delays.datasets.custom_datasets import CustomDataset

lbl = [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]

imgs = [[1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 
         [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]]

num_samples = 15000

data = np.zeros((num_samples, 1, 10))
labels = np.zeros((num_samples, 10))

for i in range(num_samples):
    cl = np.random.randint(4)
    data[i] = 0.2*np.random.rand(10) + imgs[cl]
    labels[i] = lbl[cl]

train_dataset = CustomDataset(data[:10000], labels[:10000])
test_dataset = CustomDataset(data[10000:], labels[10000:])

dataset_dict = train_dataset.get_train_attributes()

target_classes = [1, 3, 8]
sequence_length = 10

conc_test_dataset = SequentialMemoryRetrievalDataset(test_dataset, sequence_length, target_classes)
conc_train_dataset = SequentialMemoryRetrievalDataset(train_dataset, sequence_length, target_classes)

train_dataset = MemoryCachedDataset(conc_train_dataset)
test_dataset = MemoryCachedDataset(conc_test_dataset)

total_time = 1
batch_size = 128

train_loader = DataLoader(train_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            drop_last=False,
                            pin_memory=True,
                            num_workers=0)

test_loader = DataLoader(test_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            drop_last=False,
                            pin_memory=True,
                            num_workers=0)

dataset_dict["num_output"] = conc_train_dataset.total_combinations
dataset_dict["num_training_samples"] = len(conc_train_dataset)
dataset_dict["time_ms"] = 1e-4
dataset_dict["dataset_name"] = "memorytask"

print(dataset_dict)

{'num_input': 10, 'num_training_samples': 10000, 'num_output': 9, 'time_ms': 0.0001, 'dataset_name': 'memorytask'}


Training without delays (50%)

In [8]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 100

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
delay = None
#delay = (45, 15)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 4), connection_type='f',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([0])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0])

[INFO] Delays o: tensor([0])
0.0001
Delta t: 1e-05 ms
mean of normal: 14.508657488628742
training dvs5_seq10_Nonememorytask10_l4_1d1.t7 for 100 epochs...
Epoch [1/100], learning_rates 0.001000, 0.100000


c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Step [26/78], Loss: 2.16212
l1_score: 0
Step [52/78], Loss: 2.13114
l1_score: 0
Step [78/78], Loss: 1.99931
l1_score: 0
Time elasped: 31.330262899398804
Test Loss: 1.9969338327646255
Avg spk_count per neuron for all 10 time-steps 0.3763216733932495
Avg spk per neuron per layer [2.021659375, 1.65419375, 1.35134375, 0.99395]
Test Accuracy of the model on the test samples: 24.340

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIST_memory
Gradient norm for 'tau_m_1': 0.0000
Gradient norm for 'tau_m_2': 0.0000
Gradient norm for 'tau_m_3': 0.0000
Gradient norm for 'tau_m_4': 0.0000
Gradient norm for 'tau_m_o': 0.0299
Gradient norm for 'f0_f1.weight': 0.0048
Gradient norm for 'f1_f2.weight': 0.0382
Gradient norm for 'f2_f3.weight': 0.0918
Gradient norm for 'f3_f4.weight': 0.3605
Gradient norm for 'f4_o.weight': 1.5827
saving max acc: 24.34
Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIS

KeyboardInterrupt: 

Delays (10,1) 77%

In [ ]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 100

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
#delay = None
delay = (10, 1)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

[INFO] Delays o: tensor([0])
0.0001
Delta t: 1e-05 ms
mean of normal: 14.508657488628742
training dvs5_seq10_(10, 1)memorytask10_l2_10d1.t7 for 100 epochs...
Epoch [1/100], learning_rates 0.001000, 0.100000
Step [26/78], Loss: 2.10740
l1_score: 0
Step [52/78], Loss: 1.98572
l1_score: 0
Step [78/78], Loss: 1.85196
l1_score: 0
Time elasped: 3.4806418418884277
Test Loss: 1.8435319036245346
Avg spk_count per neuron for all 10 time-steps 1.1929515600204468
Avg spk per neuron per layer [2.602034375, 2.169771875]
Test Accuracy of the model on the test samples: 29.940

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIST_memory
Gradient norm for 'tau_m_1': 0.0000
Gradient norm for 'tau_m_2': 0.0000
Gradient norm for 'tau_m_o': 0.0307
Gradient norm for 'f0_f1.weight': 0.0493
Gradient norm for 'f1_f2.weig

KeyboardInterrupt: 

Recurrent: 78%

In [10]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 20

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
delay = None
#delay = (50, 1)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='r',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([0])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0])

[INFO] Delays o: tensor([0])
0.0001
Delta t: 1e-05 ms
mean of normal: 14.508657488628742
training dvs5_seq10_Nonememorytask10_l2_1d1.t7 for 20 epochs...
Epoch [1/20], learning_rates 0.001000, 0.100000
Step [26/78], Loss: 2.13248
l1_score: 0
Step [52/78], Loss: 2.01584
l1_score: 0
Step [78/78], Loss: 1.93346
l1_score: 0
Time elasped: 4.2424328327178955
Test Loss: 1.9097917675971985
Avg spk_count per neuron for all 10 time-steps 1.0870429277420044
Avg spk per neuron per layer [2.345515625, 2.00265625]
Test Accuracy of the model on the test samples: 27.460

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIST_memory
Gradient norm for 'tau_m_1': 0.0000
Gradient norm for 'tau_m_2': 0.0000
Gradient norm for 'tau_m_o': 0.0399
Gradient norm for 'f0_f1.weight': 0.0464
Gradient norm for 'f1_f1.weight': 0.0593
Gradient norm for 'f1_f2.weight': 0.4348
Gradient

Delays (10,4): 74%

In [12]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 20

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
#delay = None
delay = (10, 4)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([0, 4, 8])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0, 4, 8])

[INFO] Delays o: tensor([0])
0.0001
Delta t: 1e-05 ms
mean of normal: 14.508657488628742
training dvs5_seq10_(10, 4)memorytask10_l2_10d4.t7 for 20 epochs...
Epoch [1/20], learning_rates 0.001000, 0.100000
Step [26/78], Loss: 2.07080
l1_score: 0
Step [52/78], Loss: 1.93441
l1_score: 0
Step [78/78], Loss: 1.97424
l1_score: 0
Time elasped: 3.926501512527466
Test Loss: 1.9555294454097747
Avg spk_count per neuron for all 10 time-steps 1.1307835578918457
Avg spk per neuron per layer [2.72014375, 1.802990625]
Test Accuracy of the model on the test samples: 24.860

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIST_memory
Gradient norm for 'tau_m_1': 0.0000
Gradient norm for 'tau_m_2': 0.0000
Gradient norm for 'tau_m_o': 0.0406
Gradient norm for 'f0_f1.weight': 0.1073
Gradient norm for 'f1_f2.weight': 0.8903
Gradient norm for 'f2_o.weight': 2

With longer sequence lenghts

In [16]:
from snn_delays.datasets.custom_datasets import CustomDataset

lbl = [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]

imgs = [[1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 
         [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]]

num_samples = 15000

data = np.zeros((num_samples, 1, 10))
labels = np.zeros((num_samples, 10))

for i in range(num_samples):
    cl = np.random.randint(4)
    data[i] = 0.2*np.random.rand(10) + imgs[cl]
    labels[i] = lbl[cl]

train_dataset = CustomDataset(data[:10000], labels[:10000])
test_dataset = CustomDataset(data[10000:], labels[10000:])

dataset_dict = train_dataset.get_train_attributes()

target_classes = [1, 3, 8]
sequence_length = 100

conc_test_dataset = SequentialMemoryRetrievalDataset(test_dataset, sequence_length, target_classes)
conc_train_dataset = SequentialMemoryRetrievalDataset(train_dataset, sequence_length, target_classes)

train_dataset = MemoryCachedDataset(conc_train_dataset)
test_dataset = MemoryCachedDataset(conc_test_dataset)

total_time = 1
batch_size = 128

train_loader = DataLoader(train_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            drop_last=False,
                            pin_memory=True,
                            num_workers=0)

test_loader = DataLoader(test_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            drop_last=False,
                            pin_memory=True,
                            num_workers=0)

dataset_dict["num_output"] = conc_train_dataset.total_combinations
dataset_dict["num_training_samples"] = len(conc_train_dataset)
dataset_dict["time_ms"] = 1e-3
dataset_dict["dataset_name"] = "memorytask"

print(dataset_dict)

{'num_input': 10, 'num_training_samples': 10000, 'num_output': 9, 'time_ms': 0.001, 'dataset_name': 'memorytask'}


Recurrent: 16%

In [17]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 20

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
delay = None
#delay = (50, 1)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='r',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([0])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0])

[INFO] Delays o: tensor([0])
0.001
Delta t: 1e-05 ms
mean of normal: 14.508657488628742
training dvs5_seq100_Nonememorytask100_l2_1d1.t7 for 20 epochs...
Epoch [1/20], learning_rates 0.001000, 0.100000
Step [26/78], Loss: 2.29481
l1_score: 0
Step [52/78], Loss: 2.19039
l1_score: 0
Step [78/78], Loss: 2.21622
l1_score: 0
Time elasped: 357.53864908218384
Test Loss: 2.212411403656006
Avg spk_count per neuron for all 100 time-steps 12.222721099853516
Avg spk per neuron per layer [27.838278125, 21.052609375]
Test Accuracy of the model on the test samples: 12.220

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIST_memory
Gradient norm for 'tau_m_1': 0.0000
Gradient norm for 'tau_m_2': 0.0000
Gradient norm for 'tau_m_o': 0.0362
Gradient norm for 'f0_f1.weight': 0.0968
Gradient norm for 'f1_f1.weight': 0.1771
Gradient norm for 'f1_f2.weight': 1.2339
Grad

Delays (full): 12%

In [18]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 20

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
#delay = None
delay = (100, 1)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

[INFO] Delays o: tensor([0])
0.001
De

KeyboardInterrupt: 

In [19]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 20

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
#delay = None
delay = (2, 1)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([0, 1])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0, 1])

[INFO] Delays o: tensor([0])
0.001
Delta t: 1e-05 ms
mean of normal: 14.508657488628742
training dvs5_seq100_(2, 1)memorytask100_l2_2d1.t7 for 20 epochs...
Epoch [1/20], learning_rates 0.001000, 0.100000
Step [26/78], Loss: 2.24239
l1_score: 0
Step [52/78], Loss: 2.22359
l1_score: 0
Step [78/78], Loss: 2.19778
l1_score: 0
Time elasped: 39.87898373603821
Test Loss: 2.2141391932964325
Avg spk_count per neuron for all 100 time-steps 10.067021369934082
Avg spk per neuron per layer [23.759353125, 16.508734375]
Test Accuracy of the model on the test samples: 12.100

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIST_memory
Gradient norm for 'tau_m_1': 0.0000
Gradient norm for 'tau_m_2': 0.0000
Gradient norm for 'tau_m_o': 0.0138
Gradient norm for 'f0_f1.weight': 0.2334
Gradient norm for 'f1_f2.weight': 2.1991
Gradient norm for 'f2_o.weight': 5.45

KeyboardInterrupt: 

In [20]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 20

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
#delay = None
delay = (50, 1)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=delay, delay_type='i', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

[INFO] Delays i: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

[INFO] Delays h: tensor([0])

[INFO] Delays o: tensor([0])
0.001
Delta t: 1e-05 ms
mean of normal: 14.508657488628742
training dvs5_seq100_(50, 1)memorytask100_l2_50d1.t7 for 20 epochs...
Epoch [1/20], learning_rates 0.001000, 0.100000
Step [26/78], Loss: 2.36547
l1_score: 0
Step [52/78], Loss: 2.20326
l1_score: 0
Step [78/78], Loss: 2.20739
l1_score: 0
Time elasped: 39.742767333984375
Test Loss: 2.2190694987773893
Avg spk_count per neuron for all 100 time-steps 12.099540710449219
Avg spk per neuron per layer 

In [22]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 20

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
#delay = None
delay = (2, 1)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='r',
    delay=delay, delay_type='i', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([0, 1])

[INFO] Delays i: tensor([0, 1])

[INFO] Delays h: tensor([0])

[INFO] Delays o: tensor([0])
0.001
Delta t: 1e-05 ms
mean of normal: 14.508657488628742
training dvs5_seq100_(2, 1)memorytask100_l2_2d1.t7 for 20 epochs...
Epoch [1/20], learning_rates 0.001000, 0.100000
Step [26/78], Loss: 2.23657
l1_score: 0
Step [52/78], Loss: 2.23390
l1_score: 0
Step [78/78], Loss: 2.16834
l1_score: 0
Time elasped: 35.51646113395691
Test Loss: 2.208192455768585
Avg spk_count per neuron for all 100 time-steps 10.24219799041748
Avg spk per neuron per layer [19.94885, 21.01994375]
Test Accuracy of the model on the test samples: 13.340

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIST_memory
Gradient norm for 'tau_m_1': 0.0000
Gradient norm for 'tau_m_2': 0.0000
Gradient norm for 'tau_m_o': 0.0935
Gradient norm for 'f0_f1.weight': 0.2520
Gradient norm for 'f1_f1.weight': 0.2425
Gradient norm for 'f1_f2.weight': 1.6552
Gra

KeyboardInterrupt: 

In [23]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 20

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
#delay = None
delay = (60, 1)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=delay, delay_type='iho', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59])

[INFO] Delays i: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59])

[INFO] Delays h: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59])

[INFO] Delays o: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23,

KeyboardInterrupt: 

In [ ]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 20

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
#delay = None
delay = (60, 1)  
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=delay, delay_type='iho', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_last', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59])

[INFO] Delays i: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59])

[INFO] Delays h: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59])

[INFO] Delays o: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23,

KeyboardInterrupt: 

In [25]:
from snn_delays.datasets.custom_datasets import CustomDataset

lbl = [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]

imgs = [[1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 
         [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]]

num_samples = 15000

data = np.zeros((num_samples, 1, 10))
labels = np.zeros((num_samples, 10))

for i in range(num_samples):
    cl = np.random.randint(4)
    data[i] = 0.2*np.random.rand(10) + imgs[cl]
    labels[i] = lbl[cl]

train_dataset = CustomDataset(data[:10000], labels[:10000])
test_dataset = CustomDataset(data[10000:], labels[10000:])

dataset_dict = train_dataset.get_train_attributes()

target_classes = [1, 3, 8]
sequence_length = 30

conc_test_dataset = SequentialMemoryRetrievalDataset(test_dataset, sequence_length, target_classes)
conc_train_dataset = SequentialMemoryRetrievalDataset(train_dataset, sequence_length, target_classes)

train_dataset = MemoryCachedDataset(conc_train_dataset)
test_dataset = MemoryCachedDataset(conc_test_dataset)

total_time = 1
batch_size = 128

train_loader = DataLoader(train_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            drop_last=False,
                            pin_memory=True,
                            num_workers=0)

test_loader = DataLoader(test_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            drop_last=False,
                            pin_memory=True,
                            num_workers=0)

dataset_dict["num_output"] = conc_train_dataset.total_combinations
dataset_dict["num_training_samples"] = len(conc_train_dataset)
dataset_dict["time_ms"] = 1e-3
dataset_dict["dataset_name"] = "memorytask"

print(dataset_dict)

{'num_input': 10, 'num_training_samples': 10000, 'num_output': 9, 'time_ms': 0.001, 'dataset_name': 'memorytask'}


In [26]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 20

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
#delay = None
delay = (30, 1)  
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_last', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

[INFO] Delays o: tensor([0])
0.001
Delta t: 3.3333333333333335e-05 ms
mean of normal: 13.304684100858815
training dvs5_seq30_(30, 1)memorytask30_l2_30d1.t7 for 20 epochs...
Epoch [1/20], learning_rates 0.001000, 0.100000
Step [26/78], Loss: 2.19670
l1_score: 0
Step [52/78], Loss: 2.18870
l1_score: 0
Step [78/78], Loss: 2.19842
l1_score: 0
Time elasped: 97.99529337882996
Test Loss: 2.1964593648910524
Avg spk_count per neuron for all 30 time-steps 3.7133076190948486
Avg spk per neuron per layer [8.825028125, 6.028203125]
Test Accuracy of the model on the test samples: 11.620

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Ch

In [27]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 20

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
delay = None
#delay = (30, 1)  
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='r',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_last', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([0])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0])

[INFO] Delays o: tensor([0])
0.001
Delta t: 3.3333333333333335e-05 ms
mean of normal: 13.304684100858815
training dvs5_seq30_Nonememorytask30_l2_1d1.t7 for 20 epochs...
Epoch [1/20], learning_rates 0.001000, 0.100000
Step [26/78], Loss: 2.18826
l1_score: 0
Step [52/78], Loss: 2.17129
l1_score: 0
Step [78/78], Loss: 2.16076
l1_score: 0
Time elasped: 20.648401737213135
Test Loss: 2.1862941086292267
Avg spk_count per neuron for all 30 time-steps 3.7816593647003174
Avg spk per neuron per layer [7.9062875, 7.22035]
Test Accuracy of the model on the test samples: 15.480

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIST_memory
Gradient norm for 'tau_m_1': 0.0000
Gradient norm for 'tau_m_2': 0.0000
Gradient norm for 'tau_m_o': 0.0481
Gradient norm for 'f0_f1.weight': 0.0343
Gradient norm for 'f1_f1.weight': 0.0580
Gradient norm for 'f1_f2.weight': 0.29

In [28]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 20

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
#delay = None
delay = (3, 1)  
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='r',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_last', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([0, 1, 2])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0, 1, 2])

[INFO] Delays o: tensor([0])
0.001
Delta t: 3.3333333333333335e-05 ms
mean of normal: 13.304684100858815
training dvs5_seq30_(3, 1)memorytask30_l2_3d1.t7 for 20 epochs...
Epoch [1/20], learning_rates 0.001000, 0.100000
Step [26/78], Loss: 2.19557
l1_score: 0
Step [52/78], Loss: 2.20186
l1_score: 0
Step [78/78], Loss: 2.19354
l1_score: 0
Time elasped: 12.638879299163818
Test Loss: 2.1995633006095887
Avg spk_count per neuron for all 30 time-steps 5.359065532684326
Avg spk per neuron per layer [9.040584375, 12.395678125]
Test Accuracy of the model on the test samples: 10.900

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIST_memory
Gradient norm for 'tau_m_1': 0.0000
Gradient norm for 'tau_m_2': 0.0000
Gradient norm for 'tau_m_o': 0.0075
Gradient norm for 'f0_f1.weight': 0.0005
Gradient norm for 'f1_f1.weight': 0.0015
Gradient norm for 